In [18]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

# #BERT
# #Positivity Score
# from transformers import pipeline
# # Allocate a pipeline for sentiment-analysis
# classifier = pipeline('sentiment-analysis')

# #Summary
# import torch
# from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
# model = T5ForConditionalGeneration.from_pretrained('t5-small')
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# device = torch.device('cpu')

# #Capitalize the Sentences
# import textwrap
# import nltk.data
# sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# import pymongo
# from pymongo import MongoClient
# import ssl

In [19]:
last_run_date = datetime.strptime("February 7, 2022 00:00 AM", "%B %d, %Y %H:%M %p")

In [20]:
def convert_date(date_string):
    # February 7, 2022 10:07 AM
    
    datetime_object = datetime.strptime(date_string, "%B %d, %Y %H:%M %p")
    return (datetime_object)

In [21]:
def collect_links(page):
    stop = False
    req = requests.get("https://www.india.com/sports/page/{}".format(page))
    if(req.status_code == 200):
        soup = BeautifulSoup(req.text, "html.parser")
        try:
            section = soup.find(class_ = "listing-cities-news")
            news_items = section.findAll(class_ = "catPgListitem")
            news_links = []
            for item in news_items:
                try:
                    formatted_date = convert_date(item.find(class_ = "byline").text.split("\n")[2].strip()[:-4])
                    if(formatted_date < last_run_date):
                        stop = True
                        break
                    news_links.append(item.find("a").attrs["href"])
                except:
                    continue
            return news_links, stop
        except:
            print("Page Error.")

In [22]:
def update_last_run_date(lastdate):
    datedata=pd.read_csv("./LastRunDate.csv")
    datedata.loc[datedata["Scraper and Model"]=="sports", "Date"]=datetime.timestamp(lastdate)
    datedata.to_csv("LastRunDate.csv", index=False)

In [23]:
def gather_news(news_links):
    news_gathered = []
    for news_link in news_links:
        req = requests.get(news_link)
        soup = BeautifulSoup(req.text, "html.parser")

        try:
            article = soup.find(class_ = "article-page")
            heading = article.find("h1").text
            date = convert_date(article.find(class_ = "authors-m").find("aside").text.split(": ")[1].strip()[:-4])
            image_wrapper = article.find(class_ = "content-wrap").figure.div.img
            image_link = image_wrapper.attrs["data-lazy-src"]
            content_body = article.find(class_ = "articleBody")
            paragraphs = content_body.findAll("p")
            content = ""
            for para in paragraphs:
                content += para.text.split("Also Read")[0]

            news_gathered.append(
                    {
                        "title": heading,
                        "link": news_link,
                        "image_link": image_link,
                        "summary": content,
                        "positivity_score": 0,
                        "date": date,
                        "genre": "sports"
                    }
                )
        except:
            continue
    return news_gathered

In [24]:
datedata=pd.read_csv("./LastRunDate.csv")
pastdate=datetime.fromtimestamp(datedata.loc[datedata["Scraper and Model"] == "sports", "Date"].values[0])
lastdate=pastdate

In [25]:
page = 1
news_gathered = []
while(True):
    output = collect_links(page)
    stop = output[1]
    news_links = output[0]
    
    news_gathered = gather_news(news_links)
    
    if(stop):
        break
    else:
        page += 1

# update_last_run_date(datetime.now())

In [26]:
len(news_gathered)

9

In [27]:
news_gathered

[{'title': 'Extramarks Becomes the Official Learning Partner of Arsenal',
  'link': 'https://www.india.com/sports/extramarks-becomes-the-official-learning-partner-of-arsenal-5227209/',
  'image_link': 'https://static.india.com/wp-content/uploads/2022/02/pjimage-2022-02-07T133949.143.jpg',
  'summary': 'New Delhi: Extramarks, a leading global provider of new-age digital learning solutions, today announced a strategic partnership with Arsenal Football Club to become its exclusive Official Learning Partner in India, South Africa, Indonesia, and the Middle East.The landmark agreement will enable Extramarks to combine football and education to drive awareness of its offerings, attract new-age learners and promote access to ‘Complete Learning Solutions’ for their holistic learning and development. As part of the multi-year collaboration, Extramarks will introduce grassroots-level learning programs, football workshops, meet and greets with the players, and other exclusive competitions and exp